In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, TimestampType
import datetime
import os

In [2]:
# import Java home
os.environ["JAVA_HOME"] = "/usr/local/opt/openjdk@11"

In [3]:
# create spark session
spark = SparkSession.builder \
    .config("spark.driver.host", "localhost") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0") \
    .appName("StreamingPipeline") \
    .master("local[2]") \
    .getOrCreate()

/Users/luiz.oliveira/opt/anaconda3/envs/kafka_exe/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [4]:
# read data from kafka
kafka_bootstrap_servers = 'localhost:9092'
kafka_topic = 'topic_streaming_data'

streaming_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .option("failOnDataLoss", "false") \
    .load()

In [5]:
streaming_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
# Estrutura do json
json_schema = StructType([
    StructField('vendedor', StructType([
        StructField('nome', StringType()),
        StructField('codigo', StringType()),
        StructField('filial', StringType())
    ])),
    StructField('filial', StructType([
        StructField('nome', StringType()),
        StructField('estado', StringType()),
        StructField('cidade', StringType())
    ])),
    StructField('produto', StructType([
        StructField('codigo', StringType()),
        StructField('nome', StringType()),
        StructField('preço', DoubleType()),
        StructField('categoria', StringType()),
        StructField('cor', StringType())
    ])),
    StructField('quantidade', IntegerType()),
    StructField('ordem_id', IntegerType()),
    StructField('timestamp', StringType())
])

In [7]:
# Decodifica o valor da mensagem para UTF-8 e converte para json
streaming_df = streaming_df.withColumn("value", F.decode(F.col("value"), 'UTF-8'))
streaming_df = streaming_df.select(F.from_json(F.col('value').cast('string'), json_schema).alias('data'),'timestamp')

In [8]:
streaming_df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- vendedor: struct (nullable = true)
 |    |    |-- nome: string (nullable = true)
 |    |    |-- codigo: string (nullable = true)
 |    |    |-- filial: string (nullable = true)
 |    |-- filial: struct (nullable = true)
 |    |    |-- nome: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |-- produto: struct (nullable = true)
 |    |    |-- codigo: string (nullable = true)
 |    |    |-- nome: string (nullable = true)
 |    |    |-- preço: double (nullable = true)
 |    |    |-- categoria: string (nullable = true)
 |    |    |-- cor: string (nullable = true)
 |    |-- quantidade: integer (nullable = true)
 |    |-- ordem_id: integer (nullable = true)
 |    |-- timestamp: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestamp_venda: string (nullable = true)



In [9]:
filtered_df = streaming_df.filter(F.col("data.produto.categoria") == 'Eletrônicos')\
                            .select(
                                'data.produto.nome',
                                'data.produto.preço',
                                'data.quantidade',
                                'data.filial.nome',
                                'data.filial.cidade',
                                'data.ordem_id',
                                'timestamp'
                            )

In [10]:
filtered_df.printSchema()

root
 |-- nome: string (nullable = true)
 |-- preço: double (nullable = true)
 |-- quantidade: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- ordem_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [11]:
# Agregação soma total, quantidade vendida e valor médio de venda
aggregated_df = filtered_df.withWatermark("timestamp", "10 minutes")\
                           .groupBy(F.window(F.col("timestamp"), "10 minutes"),"cidade").agg(F.sum( 
                                                            F.col("preço")*F.col("quantidade")
                                                            ).alias("total"),
                                                            F.sum("quantidade").alias("quantidade_vendida"),
                           )

aggregated_df = aggregated_df.withColumn("valor_medio_de_venda", F.col("total")/F.col("quantidade_vendida"))

In [12]:
aggregated_df.printSchema()

root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- cidade: string (nullable = true)
 |-- total: double (nullable = true)
 |-- quantidade_vendida: long (nullable = true)
 |-- valor_medio_de_venda: double (nullable = true)



In [13]:
# Adiciona colunas de ano, mês, dia e hora
aggregated_df = aggregated_df.withColumn("year", F.year(F.col("window.start")))
aggregated_df = aggregated_df.withColumn("month",F.month(F.col("window.start")))
aggregated_df = aggregated_df.withColumn("day", F.dayofmonth(F.col("window.start")))
aggregated_df = aggregated_df.withColumn("hour", F.hour(F.col("window.start")))

In [14]:
# Define os path de output e checkpoint
output_path = os.getcwd().split('stream_pipeline')[0] + "stream_pipeline/output/topic_streaming_data/ingestao_manual"
checkpoint_path = os.getcwd().split('stream_pipeline')[0] + "stream_pipeline/checkpoint_path/topic_streaming_data/ingestao_manual"

In [15]:
# Savar dados em parquet
query = aggregated_df.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", output_path) \
    .option("checkpointLocation", checkpoint_path) \
    .partitionBy("year", "month", "day", "hour") \
    .trigger(processingTime='1 minute') \
    .start()

In [16]:
print(query.status)

{'message': 'Waiting for next trigger', 'isDataAvailable': False, 'isTriggerActive': False}


In [17]:
# Ler dados em parquet
df = spark.read.format('delta').parquet('/Users/luiz.oliveira/eng_data/stream_pipeline/output/topic_streaming_data/ingestao_manual')

In [18]:
df.show(truncate=False)

+------------------------------------------+--------------+-------+------------------+--------------------+----+-----+---+----+
|window                                    |cidade        |total  |quantidade_vendida|valor_medio_de_venda|year|month|day|hour|
+------------------------------------------+--------------+-------+------------------+--------------------+----+-----+---+----+
|{2024-01-04 23:10:00, 2024-01-04 23:20:00}|Curitiba      |2000.0 |1                 |2000.0              |2024|1    |4  |23  |
|{2024-01-04 23:10:00, 2024-01-04 23:20:00}|São Paulo     |1500.0 |1                 |1500.0              |2024|1    |4  |23  |
|{2024-01-04 23:20:00, 2024-01-04 23:30:00}|Florianópolis |37500.0|22                |1704.5454545454545  |2024|1    |4  |23  |
|{2024-01-04 23:20:00, 2024-01-04 23:30:00}|Belo Horizonte|19000.0|12                |1583.3333333333333  |2024|1    |4  |23  |
|{2024-01-04 23:20:00, 2024-01-04 23:30:00}|Rio de Janeiro|7500.0 |4                 |1875.0            